In [ ]:
## ===== SABER — Bloco 2: run (reset + servidor + GO/STOP + envio condicional) =====
# Rode este bloco sempre que quiser iniciar/retomar a operação.
# IMPORTANTE: rode o Bloco 1 antes (ele define mmio e helpers).

import socket
import time
import select

# --- checagem rápida de pré-requisitos do Bloco 1 ---
try:
    READ_PERIOD_S
    read_56_words
    words_to_hex_be
    pulse_reset
    set_go
    pulse_go
except NameError as e:
    raise AssertionError("Rode o Bloco 1 (setup) antes deste Bloco 2.") from e

# ---------- CONFIG DE REDE ----------
SERVER_IP        = "192.168.1.89"   # IP do PC/servidor
SERVER_PORT      = 5000             # porta TCP do servidor
DEFAULT_PULSE_MS = 20
ZERO_HEX         = "0" * 448        # 56 words * 4 bytes = 224 bytes -> 448 hex chars
PRINT_PACKET     = True             # << ligar/desligar impressão do pacote enviado

def connect_with_retries(ip, port, retry_s=2.0):
    while True:
        try:
            s = socket.create_connection((ip, port), timeout=5.0)
            s.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)
            s.setsockopt(socket.SOL_SOCKET, socket.SO_KEEPALIVE, 1)
            print(f"[NET] Connected to {ip}:{port} from {s.getsockname()[0]}")
            try:
                s.sendall(b"HELLO SABER_FPGA\n")
            except Exception:
                pass
            return s
        except OSError as e:
            print(f"[NET] Connect failed ({e}). Retrying in {retry_s:.1f}s…")
            time.sleep(retry_s)

def handle_server_cmd(line: str, state: dict):
    """
    Manipula comandos e retorna flags que o laço principal usa.
    Retorna dict com possíveis chaves: {'go':True}, {'reset':True}, {'stop':True}, {'pulse':ms}
    """
    cmd = line.strip()
    if not cmd:
        return {}
    u = cmd.upper().split()
    out = {}

    try:
        if u[0] == "GO":
            if len(u) == 1:
                out['pulse'] = DEFAULT_PULSE_MS
            elif u[1] == "1":
                set_go(1)
            elif u[1] == "0":
                set_go(0)
            out['go'] = True

        elif u[0] == "PULSE" and len(u) == 2 and u[1].isdigit():
            out['pulse'] = int(u[1]); out['go'] = True

        elif u[0] == "RESET":
            if len(u) == 1:
                out['reset'] = True
            elif len(u) == 2 and u[1].isdigit():
                out['reset']    = True
                out['reset_ms'] = int(u[1])

        elif u[0] == "STOP":
            out['stop'] = True

    except Exception as e:
        print(f"[CMD] error: {e}")

    return out

# ---------- Impressão do pacote (1792 bits) ----------
def _print_packet_words(hex1792: str):
    words = [hex1792[i:i+8] for i in range(0, len(hex1792), 8)]
    print("WORDS (56 x 32b):")
    for i, w in enumerate(words):
        print(f"  W{i:02d}: 0x{w}")
    print()

def _print_packet_bytes(hex1792: str, per_row: int = 16):
    bs = [hex1792[i:i+2] for i in range(0, len(hex1792), 2)]
    print(f"BYTES ({len(bs)} bytes):")
    for off in range(0, len(bs), per_row):
        row = bs[off:off+per_row]
        print(f"  {off:03d}: " + " ".join(row))
    print()

def print_packet(hex1792: str):
    _print_packet_words(hex1792)
    _print_packet_bytes(hex1792, per_row=16)

# ====== Reset antes de iniciar (opcional) ======
print("[SYS] Resetando o sistema (pré-run)…")
pulse_reset(50)
set_go(0)

# ====== Conexão TCP ======
sock = connect_with_retries(SERVER_IP, SERVER_PORT)
recv_buf = b""

# ====== Máquina de estados ======
state   = {'mode': 'IDLE'}  # 'IDLE' esperando GO | 'RUN' enviando frames
last_hex = None
seq      = 0

try:
    while True:
        # 1) Recebe comandos do servidor sem bloquear a coleta
        try:
            rlist, _, _ = select.select([sock], [], [], READ_PERIOD_S)
            if rlist:
                data = sock.recv(4096)
                if not data:
                    raise OSError("server closed")
                recv_buf += data
                while b"\n" in recv_buf:
                    line, recv_buf = recv_buf.split(b"\n", 1)
                    actions = handle_server_cmd(line.decode("ascii", errors="ignore"), state)

                    # Executa ações
                    if 'pulse' in actions:
                        set_go(1); time.sleep(actions['pulse']/1000.0); set_go(0)
                    if actions.get('reset'):
                        ms = actions.get('reset_ms', 50)
                        pulse_reset(ms); set_go(0)
                        state['mode'] = 'IDLE'; last_hex = None
                    if actions.get('stop'):
                        pulse_reset(50); set_go(0)
                        state['mode'] = 'IDLE'; last_hex = None
                    if actions.get('go'):
                        if state['mode'] != 'RUN':
                            state['mode'] = 'RUN'
                            last_hex = None

        except (BlockingIOError, InterruptedError):
            pass
        except OSError as e:
            print(f"[NET] Receive failed ({e}). Reconnecting…")
            try: sock.close()
            except Exception: pass
            sock = connect_with_retries(SERVER_IP, SERVER_PORT)
            recv_buf = b""
            continue

        # 2) Operação por modo
        if state['mode'] == 'IDLE':
            continue

        # state['mode'] == 'RUN'
        words   = read_56_words()
        hex1792 = words_to_hex_be(words)  # 56 x 32b -> 224 bytes -> 448 hex

        # Envia somente se mudou e for ≠ 0
        if hex1792 != ZERO_HEX and hex1792 != last_hex:
            ts = time.strftime("%Y-%m-%d %H:%M:%S")
            N  = len(hex1792) // 2  # 224 bytes
            msg = f"[{ts}] seq={seq} N={N} HEX:{hex1792}\n"
            try:
                sock.sendall(msg.encode("ascii"))
                print("mensagem enviada")
                if PRINT_PACKET:
                    print_packet(hex1792)
                last_hex = hex1792
            except OSError as e:
                print(f"[NET] Send failed ({e}). Reconnecting…")
                try: sock.close()
                except Exception: pass
                sock = connect_with_retries(SERVER_IP, SERVER_PORT)

        seq += 1

except KeyboardInterrupt:
    print("\nStopped by user.")
finally:
    try: sock.close()
    except Exception: pass
